# 재구축 데이터셋 Scratch



In [203]:
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets, models
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset
from PIL import Image
import numpy as np
from tqdm import tqdm

In [204]:
# 구축된 .npy파일을 Pytorch DataLoader을 사용할 수 있도록 CUSTOM DATASET을 만듬.
import numpy as np
from google.colab import drive
from sklearn.model_selection import train_test_split

default_path = "/content/drive/MyDrive/인공지능 수업/final/"

CUB200_TYPE_TRAIN = 1
CUB200_TYPE_TEST = 2
CUB200_TYPE_SUBMIT = 3

drive.mount('/content/drive')
class CUB200(data.Dataset):

    def __init__(self, type, transform = None):
        super(CUB200, self).__init__()
        """
        type : int = 1, 2, 3
        """

        original_train_data = np.load(default_path + 'train_image.npy')
        original_train_label = np.load(default_path + 'train_label.npy')

        train_data, test_data, train_label, test_label = train_test_split(
            original_train_data,
            original_train_label,
            test_size = 0.3,
            random_state = 1)
        
        if type == CUB200_TYPE_TRAIN:
          self.image = train_data
          self.label = train_label
        elif type == CUB200_TYPE_TEST:
          self.image = test_data
          self.label = test_label
        elif type == CUB200_TYPE_SUBMIT:
          self.image = np.load(default_path + 'test_image.npy')
          self.label = None
        
        self.transform = transform

    def __getitem__(self, index):
        img, target = self.image[index], self.label[index]
        img = Image.fromarray(img)

        if self.transform is not None:
            img = self.transform(img)

        return img, target

    def __len__(self):
        return len(self.image)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [205]:
trainCUB = CUB200(CUB200_TYPE_TRAIN)
print(trainCUB.image.shape)
print(trainCUB.label.shape)

testCUB = CUB200(CUB200_TYPE_TEST)
print(testCUB.image.shape)
print(testCUB.label.shape)
print(np.max(testCUB.label), np.min(testCUB.label))

testCUB = CUB200(CUB200_TYPE_SUBMIT)
print(testCUB.image.shape)

(626, 256, 256, 3)
(626,)
(269, 256, 256, 3)
(269,)
49 0
(500, 256, 256, 3)


In [206]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [207]:
# train_data에만 data augmentaion을 적용
transform_train = transforms.Compose([
        transforms.RandomCrop(224),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

transform_result = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

In [208]:
# CUSTOM DATASET을 이용하여 train_loader, test_loader을 구축

batch_size = 25

train_loader = torch.utils.data.DataLoader(
    dataset = CUB200(CUB200_TYPE_TRAIN, transform = transform_train),
    batch_size = batch_size,
    shuffle = True
)

test_loader = torch.utils.data.DataLoader(
    dataset = CUB200(CUB200_TYPE_TEST, transform = transform_test),
    batch_size = batch_size,
    shuffle = False
)

In [209]:
def training_model(model, criterion, optimizer, scheduler, num_epochs = 25):


    for epoch in range(num_epochs):
        scheduler.step()

        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            if i % 20 == 19:
                print('[%d, %5d] loss: %.7f' %
                    (epoch + 1, (i + 1), running_loss / 20))
                running_loss = 0.0
        
        train_correct = 0
        train_total = 0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            inputs = inputs.squeeze()
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)

            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()

        print('[%d epoch] Accuracy of the network on the train images: %d %%' %
              (epoch + 1, 100 * train_correct / train_total))
        
    print("End Training do it eval_accuracy")
    return model

In [210]:
def eval_accuracy(model):
    class_correct = list(0. for i in range(50))
    class_total = list(0. for i in range(50))

    correct = 0
    total = 0
    
    model.eval()
    with torch.no_grad():
        for i, data in enumerate(test_loader, 0):
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
                    
            for i in range(labels.shape[0]):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1
                
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

    print('Accuracy of the network on test images: %d %%' % (
        100 * correct / total))            
                
    return 

In [211]:
model_ft = models.resnext50_32x4d(pretrained=True)
for param in model_ft.parameters():
  param.requires_grad = False
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Sequential(
        nn.Linear(model_ft.fc.in_features, 50),
        #nn.Linear(512, 256),
        #nn.Linear(256, 50)
)
print(model_ft)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1

In [212]:
num_epochs = 16
model_ft.to(device)
criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model_ft.parameters(), lr = 0.001)
optimizer = optim.SGD(model_ft.parameters(), lr=0.075, momentum=0.9)
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 16, gamma = 0.1)

In [213]:
model_ft = training_model(model_ft, criterion, optimizer, lr_scheduler, num_epochs)

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


[1,    20] loss: 6.6573304


RuntimeError: ignored

In [ ]:
eval_accuracy(model_ft)

In [ ]:
def getresult(model):
    model.eval()
    result=[]

    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            for output in outputs:
              result.append(F.softmax(output, dim=0))
    return result